In [13]:
import os
import pandas as pd

def parse_all_xlsx_files(folder_path):
    all_dfs = []

    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Check if the file has a .xlsx extension
            if file.endswith('.xlsx'):
                file_path = os.path.join(root, file)

                # Try to read the Excel file into a DataFrame
                try:
                    df = pd.read_excel(file_path)
                    
                    # Store both the file name and the DataFrame in a tuple
                    result = (file, df)
                    all_dfs.append(result)
                except Exception as e:
                    print(f"Error reading file {file}: {e}")

    return all_dfs

# Specify the folder path containing Excel files
folder_path = 'models'

# Call the function to parse all Excel files in the folder and its subfolders
list_of_dfs = parse_all_xlsx_files(folder_path)

In [13]:
import matplotlib.pyplot as plt

# Define a sequence of markers
markers = ['o', '^', 's', 'D', 'v', '<', '>', 'p', '*', 'h']

# Create an empty list to store legend handles and labels
legend_handles = []

# Assuming list_of_dfs is already populated
for d in range(len(list_of_dfs)):
    index_to_plot = d
    
    df_to_plot = list_of_dfs[index_to_plot][1]
    file_name = list_of_dfs[index_to_plot][0]

    # Split the file name by underscore and get the second part
    entity = file_name.split('_')[1]

    # Split the entity by dot and get the first part
    parsed_entity = entity.split('.')[0]

    # Set a range for the y-axis to avoid extreme values
    y_axis_lower_limit = -0.05  # Set your desired lower limit
    y_axis_upper_limit = 1.05   # Set your desired upper limit

    # Filter values within the specified range
    filtered_df = df_to_plot[(df_to_plot['Overall'] >= y_axis_lower_limit) & (df_to_plot['Overall'] <= y_axis_upper_limit)]

    # Set 'Name' column as the index
    filtered_df.set_index('Name', inplace=True)

    # Get the marker for the current plot
    current_marker = markers[d % len(markers)]

    # Create a scatter plot using 'Name' as x-axis labels and the current marker
    scatter = plt.scatter(filtered_df.index, filtered_df['Overall'], label=f'VM of {parsed_entity}', marker=current_marker)

    # Add zebra-like background
    for i, index_name in enumerate(filtered_df.index):
        color = 'lightgray' if i % 2 == 0 else 'white'
        # Adjust the span to cover the entire area between x-axis ticks
        plt.axvspan(i - 0.5, i + 0.5, facecolor=color, alpha=0.3, zorder=-1)

    # Append the scatter plot handle to the legend_handles list
    legend_handles.append(scatter)

# Draw horizontal red lines at y=0 and y=1
plt.axhline(0, color='red', linestyle='--', linewidth=1, label='y=0')
plt.axhline(1, color='red', linestyle='--', linewidth=1, label='y=1')

# Set labels and title
plt.title('Variance Metric (VM) score depending on the characteristic')
plt.xlabel('Name of characteristic')
plt.ylabel('Variance Metric score')

# Rotate the figure 90 degrees clockwise
plt.xticks(rotation=90)

# Add legend outside the plot
plt.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(1, 1))

# Set a white background
plt.gca().set_facecolor('white')

# Adjust figure size if needed
# Adjust figure size if needed
plt.gcf().set_size_inches(12, 8)

# Save the figure with an explicit bounding box
plt.savefig('VM_all_checked.eps', bbox_inches='tight', format='eps', dpi=300)
# Show the plot
plt.show()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [14]:
# Initialize an empty dictionary
result_dict = {}

# Iterate through the list_of_dfs
for i in range(len(list_of_dfs)):
    # Extract relevant information
    entity = list_of_dfs[i][0].split('_')[1].split('.')[0]
    names = list_of_dfs[i][1]['Name'].tolist()
    r2_values = list_of_dfs[i][1]['MPE'].tolist()

    # Create a dictionary for each name
    for j in range(len(names)):
        name = names[j]
        r2_value = r2_values[j]

        # If the name is not already a key in result_dict, add it
        if name not in result_dict:
            result_dict[name] = {}

        # Add the entity and its corresponding R2 value to the name's dictionary
        result_dict[name][entity] = r2_value

# Print or use the dictionary as needed
print(result_dict)

{'Thermal conductivity': {'AdaBoost': 274.6851134251215, 'Bagging': 27.16293718711307, 'Decision Tree': 30.2056410971164, 'Elastic Net': 523.07618514004, 'Gradient Boosting': 37.44665552099447, 'KNeighborsRegressor': 77.14452499927133, 'Lasso': 523.07618514004, 'Linear Regression': 9132306659914.848, 'Random Forest': 45.5947209432056, 'SVR': 122.1966307751935, 'XGBoost': 34.4758821048127}, 'Dynamic mechanical properties loss tangent': {'AdaBoost': 1656.223144668726, 'Bagging': 345.6723685207627, 'Decision Tree': 76.94106729604854, 'Elastic Net': 767.8460581358668, 'Gradient Boosting': 308.5012320816782, 'KNeighborsRegressor': 193.7576749525487, 'Lasso': 767.8460581358668, 'Linear Regression': 806893159597039.8, 'Random Forest': 270.7179611626937, 'SVR': 249.6291848729725, 'XGBoost': 181.457457718336}, 'Thermal decomposition temperature': {'AdaBoost': 8.21525739462773, 'Bagging': 3.363775456029029, 'Decision Tree': 2.361809401032059, 'Elastic Net': 5.290218392892728, 'Gradient Boosting'

In [16]:
import pandas as pd

# Create an empty DataFrame to store the results
table_data = {'Charactristic': [], 'Regressor': [], 'MPE': []}
for high_level_key, entity_dict in result_dict.items():
    # Find the lower-level key with the max R2 value
    max_entity = min(entity_dict, key=entity_dict.get)
    max_r2_value = entity_dict[max_entity]

    # Append data to the table_data dictionary
    table_data['Charactristic'].append(high_level_key)
    table_data['Regressor'].append(max_entity)
    table_data['MPE'].append(max_r2_value)

# Create a DataFrame from the table_data
result_table = pd.DataFrame(table_data)

# Display the result_table
result_table

Charactristic      Regressor         MPE
0                         Thermal conductivity        Bagging   27.162937
1   Dynamic mechanical properties loss tangent  Decision Tree   76.941067
2            Thermal decomposition temperature        XGBoost    2.248982
3   Hansen parameter delta-h: hydrogen bonding        XGBoost    2.561384
4                             Flexural modulus  Decision Tree   10.428773
..                                         ...            ...         ...
63                       Fiber tensile modulus            SVR   34.779701
64                  Crystallization kinetics r        XGBoost  295.333506
65                               Contact angle        XGBoost    3.388830
66                             Surface tension  Decision Tree    0.055845
67                             Specific volume        XGBoost    2.750313

[68 rows x 3 columns]

In [18]:
result_table_MPE = pd.read_csv('result_table_MPE.csv')
result_table_MPE

Unnamed: 0                               Charactristic      Regressor  \
0            0                        Thermal conductivity        Bagging   
1            1  Dynamic mechanical properties loss tangent  Decision Tree   
2            2           Thermal decomposition temperature        XGBoost   
3            3  Hansen parameter delta-h: hydrogen bonding        XGBoost   
4            4                            Flexural modulus  Decision Tree   
..         ...                                         ...            ...   
63          63                       Fiber tensile modulus            SVR   
64          64                  Crystallization kinetics r        XGBoost   
65          65                               Contact angle        XGBoost   
66          66                             Surface tension  Decision Tree   
67          67                             Specific volume        XGBoost   

           MPE  
0    27.162937  
1    76.941067  
2     2.248982  
3     2.561384  
4    10.428773  
..         ...  
63   34.779701  
64  295.333506  
65    3.388830  
66    0.055845  
67    2.750313  

[68 rows x 4 columns]

In [23]:
best_MPE = result_table_MPE[result_table_MPE['MPE']<50]
best_MPE

Unnamed: 0                                   Charactristic  \
0            0                            Thermal conductivity   
2            2               Thermal decomposition temperature   
3            3      Hansen parameter delta-h: hydrogen bonding   
4            4                                Flexural modulus   
5            5                     Vicat softening temperature   
7            7                    Pvt relation specific volume   
11          11                 Lc phase transition temperature   
12          12     Dynamic flexural properties storage modulus   
13          13               Thermal decomposition weight loss   
14          14                             Melting temperature   
15          15                  Hansen parameter delta p polar   
19          19                             Elongation at yield   
20          20           Dynamic shear properties loss tangent   
22          22                         Cohesive energy density   
23          23                    Glass transition temperature   
25          25                      Crystallization kinetics n   
27          27                                         Density   
32          32               Flexural stress strength at break   
38          38              Isothermal weight loss temperature   
39          39                            Solubility parameter   
40          40           Deflection temperature under load hdt   
41          41                       Specific heat capacity cp   
42          42          Fiber tensile stress strength at break   
43          43                         Brittleness temperature   
44          44                Tensile stress strength at yield   
47          47                                Refractive index   
49          49                                    Oxygen index   
53          53                             Thermal diffusivity   
55          55                              Radius of gyration   
57          57                     Crystallization temperature   
60          60  Hansen parameter delta-d: dispersion component   
61          61                           Softening temperature   
62          62                          Dielectric constant ac   
63          63                           Fiber tensile modulus   
65          65                                   Contact angle   
66          66                                 Surface tension   
67          67                                 Specific volume   

            Regressor        MPE  
0             Bagging  27.162937  
2             XGBoost   2.248982  
3             XGBoost   2.561384  
4       Decision Tree  10.428773  
5               Lasso   0.453383  
7                 SVR   0.014522  
11            Bagging   3.015693  
12            XGBoost  34.925791  
13      Decision Tree  14.646020  
14      Decision Tree   1.054177  
15                SVR   0.449479  
19            XGBoost  14.025028  
20            XGBoost   3.836445  
22  Gradient Boosting   0.959539  
23              Lasso   1.229773  
25           AdaBoost   3.853924  
27                SVR   0.504447  
32            XGBoost  30.770828  
38            Bagging   0.134962  
39        Elastic Net   0.043511  
40      Random Forest   3.996161  
41                SVR   9.592151  
42  Gradient Boosting   1.095852  
43              Lasso   1.200470  
44      Decision Tree   9.632560  
47      Decision Tree   0.912143  
49            Bagging  12.241440  
53            Bagging   4.132031  
55  Gradient Boosting   4.406617  
57  Gradient Boosting   6.296890  
60           AdaBoost   0.240979  
61            Bagging  20.728498  
62                SVR   2.813924  
63                SVR  34.779701  
65            XGBoost   3.388830  
66      Decision Tree   0.055845  
67            XGBoost   2.750313

In [35]:
final_1 = pd.read_csv('best_res.csv')
#final_1 = final_1.sort_values(by='Max $R^2$', ascending=False)

In [36]:
final_1

Unnamed: 0                               Charactristic  \
0           11                Glass transition temperature   
1            1           Thermal decomposition temperature   
2            7                         Melting temperature   
3           17                     Intrinsic viscosity eta   
4           30                             Specific volume   
5           14                                     Density   
6           15                         Elongation at break   
7            6             Lc phase transition temperature   
8           28                       Softening temperature   
9           23                            Refractive index   
10          27                 Crystallization temperature   
11          29                             Surface tension   
12          19                        Solubility parameter   
13          10                     Cohesive energy density   
14           0  Dynamic mechanical properties loss tangent   
15          18          Isothermal weight loss temperature   
16          16                 Isothermal weight loss time   
17          24                                Oxygen index   
18          12    Dynamic shear properties storage modulus   
19          13                     Heat of crystallization   
20          20       Deflection temperature under load hdt   
21          21      Fiber tensile stress strength at break   
22           3                 Vicat softening temperature   
23          22                     Brittleness temperature   
24          26                         Thermal diffusivity   
25           5                    Water vapor transmission   
26           8              Hansen parameter delta p polar   
27           2  Hansen parameter delta-h: hydrogen bonding   
28           9                  Crystallization kinetics k   
29           4                Pvt relation specific volume   
30          25                       Pvt relation pressure   

              Regressor  Max $R^2$  NMSE  Data Size_x  
0         Random Forest       0.88  0.12         8092  
1         Random Forest       0.73  0.27         6325  
2         Random Forest       0.71  0.29         3844  
3     Gradient Boosting       0.74  0.26         1978  
4               XGBoost       0.71  0.29         1739  
5               XGBoost       0.56  0.44         1739  
6     Gradient Boosting       0.55  0.45         1139  
7         Random Forest       0.79  0.21          961  
8         Random Forest       0.68  0.32          777  
9               XGBoost       0.73  0.27          685  
10        Random Forest       0.69  0.31          457  
11              Bagging       0.59  0.41          348  
12              XGBoost       0.77  0.23          324  
13              XGBoost       0.82  0.18          324  
14    Gradient Boosting       0.52  0.48          301  
15              XGBoost       0.97  0.03          273  
16              XGBoost       0.86  0.14          228  
17              XGBoost       0.65  0.35          176  
18  KNeighborsRegressor       0.51  0.49          141  
19        Random Forest       0.65  0.35          124  
20        Random Forest       0.61  0.39           99  
21        Decision Tree       0.63  0.37           91  
22    Gradient Boosting       0.67  0.33           82  
23  KNeighborsRegressor       0.67  0.33           81  
24              Bagging       0.94  0.06           80  
25                  SVR       0.73  0.27           73  
26              Bagging       0.90  0.10           59  
27             AdaBoost       0.59  0.41           59  
28              XGBoost       0.97  0.03           59  
29        Decision Tree       0.78  0.22           56  
30        Decision Tree       0.96  0.04           53

In [42]:
final_1 = pd.merge(final_1, best_MPE[['Charactristic', 'MPE']], on='Charactristic', how='left')
final_1

Unnamed: 0                               Charactristic  \
0           11                Glass transition temperature   
1            1           Thermal decomposition temperature   
2            7                         Melting temperature   
3           17                     Intrinsic viscosity eta   
4           30                             Specific volume   
5           14                                     Density   
6           15                         Elongation at break   
7            6             Lc phase transition temperature   
8           28                       Softening temperature   
9           23                            Refractive index   
10          27                 Crystallization temperature   
11          29                             Surface tension   
12          19                        Solubility parameter   
13          10                     Cohesive energy density   
14           0  Dynamic mechanical properties loss tangent   
15          18          Isothermal weight loss temperature   
16          16                 Isothermal weight loss time   
17          24                                Oxygen index   
18          12    Dynamic shear properties storage modulus   
19          13                     Heat of crystallization   
20          20       Deflection temperature under load hdt   
21          21      Fiber tensile stress strength at break   
22           3                 Vicat softening temperature   
23          22                     Brittleness temperature   
24          26                         Thermal diffusivity   
25           5                    Water vapor transmission   
26           8              Hansen parameter delta p polar   
27           2  Hansen parameter delta-h: hydrogen bonding   
28           9                  Crystallization kinetics k   
29           4                Pvt relation specific volume   
30          25                       Pvt relation pressure   

              Regressor  Max $R^2$  NMSE  Data Size_x      MPE_x      MPE_y  
0         Random Forest       0.88  0.12         8092   1.229773   1.229773  
1         Random Forest       0.73  0.27         6325   2.248982   2.248982  
2         Random Forest       0.71  0.29         3844   1.054177   1.054177  
3     Gradient Boosting       0.74  0.26         1978        NaN        NaN  
4               XGBoost       0.71  0.29         1739   2.750313   2.750313  
5               XGBoost       0.56  0.44         1739   0.504447   0.504447  
6     Gradient Boosting       0.55  0.45         1139        NaN        NaN  
7         Random Forest       0.79  0.21          961   3.015693   3.015693  
8         Random Forest       0.68  0.32          777  20.728498  20.728498  
9               XGBoost       0.73  0.27          685   0.912143   0.912143  
10        Random Forest       0.69  0.31          457   6.296890   6.296890  
11              Bagging       0.59  0.41          348   0.055845   0.055845  
12              XGBoost       0.77  0.23          324   0.043511   0.043511  
13              XGBoost       0.82  0.18          324   0.959539   0.959539  
14    Gradient Boosting       0.52  0.48          301        NaN        NaN  
15              XGBoost       0.97  0.03          273   0.134962   0.134962  
16              XGBoost       0.86  0.14          228        NaN        NaN  
17              XGBoost       0.65  0.35          176  12.241440  12.241440  
18  KNeighborsRegressor       0.51  0.49          141        NaN        NaN  
19        Random Forest       0.65  0.35          124        NaN        NaN  
20        Random Forest       0.61  0.39           99   3.996161   3.996161  
21        Decision Tree       0.63  0.37           91   1.095852   1.095852  
22    Gradient Boosting       0.67  0.33           82   0.453383   0.453383  
23  KNeighborsRegressor       0.67  0.33           81   1.200470   1.200470  
24              Bagging       0.94  0.06           80   4.132031   4.

In [57]:
#final_1.drop(['NMSE'], axis=1, inplace=True)#to_csv('best_res.csv')

#final_1 = pd.read_excel('best_res.xlsx')
final_1.to_csv('best_res.csv')

In [56]:
final_1['MPE'] = final_1['MPE'].round(2)

In [15]:
vm = result_table[result_table['Max VM'] > 0.5]
vm

Charactristic            Regressor  \
0                                 Elongation at yield    Gradient Boosting   
3                        Glass transition temperature        Random Forest   
5                                 Melting temperature        Random Forest   
13                 Isothermal weight loss temperature          Elastic Net   
14                  Fiber tensile elongation at break  KNeighborsRegressor   
17                               Solubility parameter    Gradient Boosting   
21                        Vicat softening temperature        Random Forest   
26                            Cohesive energy density        Random Forest   
28             Fiber tensile stress strength at break                  SVR   
29                                       Oxygen index              Bagging   
30                           Pvt relation temperature             AdaBoost   
31                        Crystallization temperature    Gradient Boosting   
33                                      Contact angle        Random Forest   
34  Crystallization kinetics half time of crystall...             AdaBoost   
35                            Heat of crystallization                Lasso   
38                  Thermal decomposition weight loss                  SVR   
39                        Isothermal weight loss time                  SVR   
41                              Pvt relation pressure             AdaBoost   
46                    Lc phase transition temperature              XGBoost   
48              Deflection temperature under load hdt              Bagging   
49                      Heat of fusion mol conversion              Bagging   
54                     Hansen parameter delta p polar              Bagging   
55                                Elongation at break        Decision Tree   
56                         Crystallization kinetics r  KNeighborsRegressor   
58                                    Surface tension              Bagging   
59                  Thermal decomposition temperature              XGBoost   
60                                        Izod impact  KNeighborsRegressor   
61                              Fiber tensile modulus             AdaBoost   
64                                 Radius of gyration              Bagging   

      Max VM  
0   0.771429  
3   0.677508  
5   0.569194  
13  1.000000  
14  0.766667  
17  0.660377  
21  0.923077  
26  0.895238  
28  0.617647  
29  0.687500  
30  0.935484  
31  0.833333  
33  0.906475  
34  0.956522  
35  0.649123  
38  0.957460  
39  0.679245  
41  0.962963  
46  0.779661  
48  0.786885  
49  0.605634  
54  0.687500  
55  0.621053  
56  0.718750  
58  0.732558  
59  0.888591  
60  0.733333  
61  0.823529  
64  0.626667

In [11]:
bests_r2 = pd.read_excel('max_r2.xlsx')

In [14]:
r2 = bests_r2[bests_r2['Max $R^2$']>0.5]
r2

Unnamed: 0                               Charactristic  \
1            1  Dynamic mechanical properties loss tangent   
2            2           Thermal decomposition temperature   
3            3  Hansen parameter delta-h: hydrogen bonding   
5            5                 Vicat softening temperature   
7            7                Pvt relation specific volume   
10          10                    Water vapor transmission   
11          11             Lc phase transition temperature   
14          14                         Melting temperature   
15          15              Hansen parameter delta p polar   
17          17                  Crystallization kinetics k   
22          22                     Cohesive energy density   
23          23                Glass transition temperature   
24          24    Dynamic shear properties storage modulus   
26          26                     Heat of crystallization   
27          27                                     Density   
30          30                         Elongation at break   
34          34                 Isothermal weight loss time   
35          35                     Intrinsic viscosity eta   
38          38          Isothermal weight loss temperature   
39          39                        Solubility parameter   
40          40       Deflection temperature under load hdt   
42          42      Fiber tensile stress strength at break   
43          43                     Brittleness temperature   
47          47                            Refractive index   
49          49                                Oxygen index   
50          50                       Pvt relation pressure   
53          53                         Thermal diffusivity   
57          57                 Crystallization temperature   
61          61                       Softening temperature   
66          66                             Surface tension   
67          67                             Specific volume   

              Regressor  Max $R^2$  
1     Gradient Boosting   0.524970  
2         Random Forest   0.725618  
3              AdaBoost   0.588484  
5     Gradient Boosting   0.668552  
7         Decision Tree   0.777472  
10                  SVR   0.728422  
11        Random Forest   0.793456  
14        Random Forest   0.707474  
15              Bagging   0.900058  
17              XGBoost   0.968771  
22              XGBoost   0.818874  
23        Random Forest   0.879994  
24  KNeighborsRegressor   0.510739  
26        Random Forest   0.646512  
27              XGBoost   0.564882  
30    Gradient Boosting   0.547713  
34              XGBoost   0.859393  
35    Gradient Boosting   0.738814  
38              XGBoost   0.967617  
39              XGBoost   0.774766  
40        Random Forest   0.614563  
42        Decision Tree   0.625683  
43  KNeighborsRegressor   0.668205  
47              XGBoost   0.725368  
49              XGBoost   0.650719  
50        Decision Tree   0.957573  
53              Bagging   0.944809  
57        Random Forest   0.692198  
61        Random Forest   0.678676  
66              Bagging   0.593994  
67              XGBoost   0.712654

In [16]:
vm_list = list(vm['Charactristic'])
r2_list = list(r2['Charactristic'])

In [17]:
# Находим названия, которые есть в all_cols, но отсутствуют в parts_list
missing_names = set(r2_list) - set(vm_list)

In [18]:
missing_names 

{'Brittleness temperature',
 'Crystallization kinetics k',
 'Density',
 'Dynamic mechanical properties loss tangent',
 'Dynamic shear properties storage modulus',
 'Hansen parameter delta-h: hydrogen bonding',
 'Intrinsic viscosity eta',
 'Pvt relation specific volume',
 'Refractive index',
 'Softening temperature',
 'Specific volume',
 'Thermal diffusivity',
 'Water vapor transmission'}

In [31]:
nonzero_counts = pd.read_excel('non_zero_var.xlsx')

In [32]:
nonzero_counts

Unnamed: 0                                Charactristic  count
0            0                         Thermal conductivity     23
1            1   Dynamic mechanical properties loss tangent    216
2            2  Dynamic compressive properties loss tangent      6
3            3            Thermal decomposition temperature   2968
4            4         Compressive stress strength at break      8
..         ...                                          ...    ...
93          93                              Surface tension    176
94          94                          Interfacial tension     13
95          95                              Specific volume   1063
96          96                     Tensile creep compliance      3
97          97               Tensile creep rupture strength      2

[98 rows x 3 columns]

In [35]:
final = vm.merge(nonzero_counts.rename(columns={'count': 'count_var'}), on='Charactristic', how='left')
final


Charactristic            Regressor  \
0                                 Elongation at yield    Gradient Boosting   
1                        Glass transition temperature        Random Forest   
2                                 Melting temperature        Random Forest   
3                  Isothermal weight loss temperature          Elastic Net   
4                   Fiber tensile elongation at break  KNeighborsRegressor   
5                                Solubility parameter    Gradient Boosting   
6                         Vicat softening temperature        Random Forest   
7                             Cohesive energy density        Random Forest   
8              Fiber tensile stress strength at break                  SVR   
9                                        Oxygen index              Bagging   
10                           Pvt relation temperature             AdaBoost   
11                        Crystallization temperature    Gradient Boosting   
12                                      Contact angle        Random Forest   
13  Crystallization kinetics half time of crystall...             AdaBoost   
14                            Heat of crystallization                Lasso   
15                  Thermal decomposition weight loss                  SVR   
16                        Isothermal weight loss time                  SVR   
17                              Pvt relation pressure             AdaBoost   
18                    Lc phase transition temperature              XGBoost   
19              Deflection temperature under load hdt              Bagging   
20                      Heat of fusion mol conversion              Bagging   
21                     Hansen parameter delta p polar              Bagging   
22                                Elongation at break        Decision Tree   
23                         Crystallization kinetics r  KNeighborsRegressor   
24                                    Surface tension              Bagging   
25                  Thermal decomposition temperature              XGBoost   
26                                        Izod impact  KNeighborsRegressor   
27                              Fiber tensile modulus             AdaBoost   
28                                 Radius of gyration              Bagging   

      Max VM  Unnamed: 0  count_var  
0   0.771429          29         49  
1   0.677508          34       6278  
2   0.569194          24       2182  
3   1.000000          56        219  
4   0.766667          54         31  
5   0.660377          58        218  
6   0.923077           9         56  
7   0.895238          33        219  
8   0.617647          62         57  
9   0.687500          70        144  
10  0.935484          76         26  
11  0.833333          79        331  
12  0.906475          92        116  
13  0.956522          78         26  
14  0.649123          38         67  
15  0.957460          23       3567  
16  0.679245          51        175  
17  0.962963          71         26  
18  0.779661          19        430  
19  0.786885          59         38  
20  0.605634          28        154  
21  0.687500          25         43  
22  0.621053          46        854  
23  0.718750          91         21  
24  0.732558          93        176  
25  0.888591           3       2968  
26  0.733333          72         23  
27  0.823529          89         40  
28  0.626667          77         45

In [4]:
final = pd.read_excel('best VM.xlsx')

In [12]:
import pandas as pd
final.to_csv('best VM.csv')

In [7]:
final = final.sort_values(by='Max VM', ascending=False)

In [11]:
final['Max VM'] = final['Max VM'].round(2)

In [80]:
with open('category_dict.json', 'r') as json_file:
    category_dict = json.load(json_file)


In [81]:
category_dict

{'Compression characteristics': ['Compressive modulus',
  'Compressive stress strength at break',
  'Compressive stress strength at yield',
  'Dynamic compressive properties storage modulus',
  'Dynamic compressive properties loss tangent',
  'Dynamic compressive properties loss modulus'],
 'Creep characteristics': ['Tensile creep compliance',
  'Tensile creep modulus',
  'Tensile creep recovery',
  'Tensile creep rupture time',
  'Tensile creep strain',
  'Flexural creep strain',
  'Tensile creep rupture strength'],
 'Dilute solution property': ['Intrinsic viscosity eta',
  'Radius of gyration',
  'Second virial coefficient',
  'Diffusion coefficient',
  'Sedimentation coefficient'],
 'Electric property': ['Dielectric constant ac',
  'Dielectric loss factor',
  'Dielectric loss tangent',
  'Electric conductivity',
  'Surface resistivity',
  'Volume resistivity'],
 'Flexural property': ['Dynamic flexural properties storage modulus',
  'Dynamic flexural properties loss modulus',
  'Dyna

In [77]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Define the sections and their keys in a dictionary
sections = {
    'Section1': {'key1': 'value1', 'key2': 'value2'},
    'Section2': {'key3': 'value3', 'key4': 'value4'},
    'Section3': {'key5': 'value5', 'key6': 'value6'},
}

# Create a Plotly subplot
fig = make_subplots()

# Add the central node 'Polymers'
fig.add_trace(go.Scatter(x=[0], y=[0], text=['Polymers'], mode='text', showlegend=False))

# Add nodes for each section
for i, (section, keys) in enumerate(sections.items(), start=1):
    x = [0.5 * (i % 2) * (-1) ** (i % 2)]  # Alternate sides
    y = [-(i // 2) * 0.2]  # Adjust vertical positions

    # Add a colorful box around each section
    fig.add_shape(
        type='rect',
        xref='x',
        yref='y',
        x0=x[0] - 0.2,
        x1=x[0] + 0.2,
        y0=y[0] - 0.1,
        y1=y[0] + 0.1 * (len(keys) + 1),
        fillcolor=f'rgba({i * 50}, {i * 30}, {i * 20}, 0.5)',
        line=dict(color='black'),
    )

    # Add section name
    fig.add_trace(go.Scatter(x=x, y=y, text=[section], mode='text', showlegend=False))

    # Add keys and values
    for j, (key, value) in enumerate(keys.items(), start=1):
        fig.add_trace(go.Scatter(x=x, y=[y[0] - 0.1 * j], text=[f"{key}: {value}"], mode='text', showlegend=False))

    # Add arrows from 'Polymers' to each section
    fig.add_trace(go.Scatter(
        x=[0, x[0]],
        y=[0, y[0]],
        mode='lines+text',
        text=[f"{key}: {value}" for key, value in keys.items()],
        line=dict(color='black'),
        showlegend=False
    ))

# Update layout for better visualization
fig.update_layout(
    hovermode='closest',
    title='Polymers and Sections',
    showlegend=False,
    margin=dict(b=0, l=0, r=0, t=30),
)

# Show the graph
fig.show()



AttributeError: 'list' object has no attribute 'items'

In [71]:
import json
import plotly.graph_objs as go

# Read JSON file
with open('Category_dict.json', 'r') as file:
    graph_json = json.load(file)

# Create Plotly graph from JSON
fig = go.Figure(graph_json)

# Show the graph
fig.show()


ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'Compression characteristics'

Did you mean "hovertemplatesrc"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        dx
   